---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [3]:
import pandas

doc = []
with open('dates.txt') as reader:
    data = pandas.Series(reader.readlines())
data.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [4]:
ZERO_OR_MORE = '*'
ONE_OR_MORE = "+"
ZERO_OR_ONE = '?'
EXACTLY_TWO = "{2}"
ONE_OR_TWO = "{1,2}"
EXACTLY_ONE = '{1}'
GROUP = r"({})"
NAMED = r"(?P<{}>{})"
CLASS = "[{}]"
NEGATIVE_LOOKAHEAD = "(?!{})"
NEGATIVE_LOOKBEHIND = "(?<!{})"
POSITIVE_LOOKAHEAD = "(?={})"
POSITIVE_LOOKBEHIND = "(?<={})"
ESCAPE = "\{}"
DIGIT = r"\d"
ONE_DIGIT  = DIGIT + EXACTLY_ONE
ONE_OR_TWO_DIGITS = DIGIT + ONE_OR_TWO
NON_DIGIT = NEGATIVE_LOOKAHEAD.format(DIGIT)
TWO_DIGITS = DIGIT + EXACTLY_TWO
THREE_DIGITS = DIGIT + "{3}"
EXACTLY_TWO_DIGITS = DIGIT + EXACTLY_TWO + NON_DIGIT
FOUR_DIGITS = DIGIT + r"{4}" + NON_DIGIT
SLASH = r"/"
OR = r'|'
LOWER_CASE = "a-z"
SPACE = "\s"
DOT = "."
DASH = "-"
COMMA = ","
PUNCTUATION = CLASS.format(DOT + COMMA + DASH)
EMPTY_STRING = ""
MONTH_SUFFIX = (CLASS.format(LOWER_CASE) + ZERO_OR_MORE
                + CLASS.format(SPACE + DOT + COMMA + DASH) + ONE_OR_TWO)
MONTH_PREFIXES = "Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec".split()
MONTHS = [month + MONTH_SUFFIX for month in MONTH_PREFIXES]
MONTHS = GROUP.format(OR.join(MONTHS))
DAY_SUFFIX = CLASS.format(DASH + COMMA + SPACE) + ONE_OR_TWO
DAYS = ONE_OR_TWO_DIGITS + DAY_SUFFIX
YEAR = FOUR_DIGITS
                                                  

In [5]:
CONTRACTED = (ONE_OR_TWO_DIGITS
              + LOWER_CASE
              + EXACTLY_TWO
              )
CONTRACTION = NAMED.format("contraction",
                           MONTHS
                           + CONTRACTED
                           + DAY_SUFFIX
                           + YEAR)


In [6]:
NO_DAY_BEHIND = NEGATIVE_LOOKBEHIND.format(DIGIT + SPACE)
NO_DAY = NAMED.format("no_day", NO_DAY_BEHIND + MONTHS + YEAR)


In [7]:
WORDS = NAMED.format("words", MONTHS + DAYS + YEAR)


In [8]:
BACKWARDS = NAMED.format("backwards", ONE_OR_TWO_DIGITS + SPACE + MONTHS + YEAR)


In [9]:
slashed = SLASH.join([ONE_OR_TWO_DIGITS,
                      ONE_OR_TWO_DIGITS,
                      EXACTLY_TWO_DIGITS])
dashed = DASH.join([ONE_OR_TWO_DIGITS,
                    ONE_OR_TWO_DIGITS,
                    EXACTLY_TWO_DIGITS])
TWENTIETH_CENTURY = NAMED.format("twentieth",
                                 OR.join([slashed, dashed]))


In [10]:
NUMERIC = NAMED.format("numeric",
                       SLASH.join([ONE_OR_TWO_DIGITS,
                                   ONE_OR_TWO_DIGITS,
                                   FOUR_DIGITS]))


In [11]:
NO_PRECEDING_SLASH = NEGATIVE_LOOKBEHIND.format(SLASH)
NO_PRECEDING_SLASH_DIGIT = NEGATIVE_LOOKBEHIND.format(CLASS.format(SLASH + DIGIT))
NO_ONE_DAY = (NO_PRECEDING_SLASH_DIGIT
              + ONE_DIGIT
              + SLASH
              + FOUR_DIGITS)
NO_TWO_DAYS = (NO_PRECEDING_SLASH
               + TWO_DIGITS
               + SLASH
               + FOUR_DIGITS)
NO_DAY_NUMERIC = NAMED.format("no_day_numeric",
                              NO_ONE_DAY
                              + OR
                              + NO_TWO_DAYS
                              )


In [12]:
CENTURY = GROUP.format('19' + OR + "20") + TWO_DIGITS
DIGIT_SLASH = DIGIT + SLASH
DIGIT_DASH = DIGIT + DASH
DIGIT_SPACE = DIGIT + SPACE
LETTER_SPACE = CLASS.format(LOWER_CASE) + SPACE
COMMA_SPACE = COMMA + SPACE
YEAR_PREFIX = NEGATIVE_LOOKBEHIND.format(OR.join([
    DIGIT_SLASH,
    DIGIT_DASH,
    DIGIT_SPACE,
    LETTER_SPACE,
    COMMA_SPACE,    
]))

YEAR_ONLY = NAMED.format("year_only",
                         YEAR_PREFIX + CENTURY
)


In [13]:
IN_PREFIX = POSITIVE_LOOKBEHIND.format(CLASS.format('iI') + 'n' + SPACE) + CENTURY
SINCE_PREFIX = POSITIVE_LOOKBEHIND.format(CLASS.format("Ss") + 'ince' + SPACE) + CENTURY
AGE = POSITIVE_LOOKBEHIND.format("Age" + SPACE + TWO_DIGITS + COMMA + SPACE) + CENTURY
AGE_COMMA = POSITIVE_LOOKBEHIND.format("Age" + COMMA + SPACE + TWO_DIGITS + COMMA + SPACE) + CENTURY
OTHERS = ['delivery', "quit", "attempt", "nephrectomy", THREE_DIGITS]
OTHERS = [POSITIVE_LOOKBEHIND.format(label + SPACE) + CENTURY for label in OTHERS]
OTHERS = OR.join(OTHERS)
LEFTOVERS_PREFIX = OR.join([IN_PREFIX, SINCE_PREFIX, AGE, AGE_COMMA]) + OR + OTHERS
LEFTOVERS = NAMED.format("leftovers", LEFTOVERS_PREFIX)


In [14]:
DATE = NAMED.format("date", OR.join([NUMERIC,
                                     TWENTIETH_CENTURY,
                                     WORDS,
                                     BACKWARDS,
                                     CONTRACTION,
                                     NO_DAY,
                                     NO_DAY_NUMERIC,
                                     YEAR_ONLY,
                                     LEFTOVERS]))


In [15]:
def extract_and_count(expression, data, name):
    """extract all matches and report the count

    Args:
     expression (str): regular expression to match
     data (pandas.Series): data with dates to extratc
     name (str): name of the group for the expression

    Returns:
     tuple (pandas.Series, int): extracted dates, count
    """
    extracted = data.str.extractall(expression)[name]
    count = len(extracted)
    print("'{}' matched {} rows".format(name, count))
    return extracted, count


In [17]:
numeric, numeric_count = extract_and_count(NUMERIC, data, 'numeric')
twentieth, twentieth_count = extract_and_count(TWENTIETH_CENTURY, data, 'twentieth')
words, words_count = extract_and_count(WORDS, data, 'words')
backwards, backwards_count = extract_and_count(BACKWARDS, data, 'backwards')
contraction_data, contraction = extract_and_count(CONTRACTION, data, 'contraction')
no_day, no_day_count = extract_and_count(NO_DAY, data, 'no_day')
no_day_numeric, no_day_numeric_count = extract_and_count(NO_DAY_NUMERIC, data,
                                                         "no_day_numeric")
year_only, year_only_count = extract_and_count(YEAR_ONLY, data, "year_only")
leftovers, leftovers_count = extract_and_count(LEFTOVERS, data, "leftovers")
found = data.str.extractall(DATE)
total_found = len(found.date)

print("Total Found: {}".format(total_found))
print("Remaining: {}".format(len(data) - total_found))
print("Discrepancy: {}".format(total_found - (numeric_count
                                              + twentieth_count
                                              + words_count
                                              + backwards_count
                                              + contraction
                                              + no_day_count
                                              + no_day_numeric_count
                                              + year_only_count
                                              + leftovers_count)))


'numeric' matched 25 rows
'twentieth' matched 100 rows
'words' matched 34 rows
'backwards' matched 69 rows
'contraction' matched 0 rows
'no_day' matched 115 rows
'no_day_numeric' matched 112 rows
'year_only' matched 15 rows
'leftovers' matched 30 rows
Total Found: 500
Remaining: 0
Discrepancy: 0


In [18]:
def add_month_date(match):
    """adds 01/01 to years

    Args:
     match (re.Match): object that only matched a 4-digit year

    Returns:
     str: 01/01/YYYY
    """
    return "01/01/" + match.group()


In [19]:
def clean(source, expression, replacement, sample=5):
    """applies the replacement to the source

    as a side-effect shows sample rows before and after

    Args:
     source (pandas.Series): source of the strings
     expression (str): regular expression to match what to replace
     replacement: function or expression to replace the matching expression
     sample (int): number of randomly chosen examples to show

    Returns:
     pandas.Series: the source with the replacement applied to it
    """
    print("Random Sample Before:")
    print(source.sample(sample))
    cleaned = source.str.replace(expression, replacement)
    print("\nRandom Sample After:")
    print(cleaned.sample(sample))
    print("\nCount of cleaned: {}".format(len(cleaned)))
    assert len(source) == len(cleaned)
    return cleaned


In [20]:
year_only_cleaned = clean(year_only,
                          YEAR_ONLY,
                          add_month_date)


Random Sample Before:
     match
486  0        1973
496  0        2006
462  0        1988
483  0        1995
497  0        2008
Name: year_only, dtype: object

Random Sample After:
     match
481  0        01/01/1974
497  0        01/01/2008
493  0        01/01/1978
486  0        01/01/1973
483  0        01/01/1995
Name: year_only, dtype: object

Count of cleaned: 15


In [45]:
leftovers_cleaned = clean(leftovers, YEAR_ONLY, add_month_date)


Random Sample Before:
     match
467  0        2011
484  0        2004
491  0        2009
465  0        1976
459  0        1998
Name: leftovers, dtype: object

Random Sample After:
     match
468  0        01/01/1997
475  0        01/01/2015
477  0        01/01/1994
492  0        01/01/1986
499  0        01/01/1980
Name: leftovers, dtype: object

Count of cleaned: 30


In [21]:
two_digit_expression = GROUP.format(ONE_OR_TWO_DIGITS) + POSITIVE_LOOKAHEAD.format(SLASH)


def two_digits(match):
    """add a leading zero if needed

    Args:
     match (re.Match): match with one or two digits

    Returns:
     str: the matched string with leading zero if needed
    """
    # for some reason the string-formatting raises an error if it's a string
    # so cast it to an int
    return "{:02}".format(int(match.group()))


In [22]:
def clean_two_digits(source, sample=5):
    """makes sure source has two-digits
    
    Args:
     source (pandas.Series): data with digit followed by slash
     sample (int): number of samples to show

    Returns:
     pandas.Series: source with digits coerced to two digits
    """
    return clean(source, two_digit_expression, two_digits, sample)


In [23]:
def clean_two_digits_isolated(source, sample=5):
    """cleans two digits that are standalone

    Args:
     source (pandas.Series): source of the data
     sample (int): number of samples to show

    Returns:
     pandas.Series: converted data
    """
    return clean(source, ONE_OR_TWO_DIGITS, two_digits, sample)


In [25]:
no_day_numeric_cleaned = clean_two_digits(no_day_numeric)
no_day_numeric_cleaned = clean(no_day_numeric_cleaned,
                               SLASH,
                               lambda m: "/01/")


Random Sample Before:
     match
452  0        3/2003
369  0        1/1978
442  0        9/1992
441  0        9/1980
379  0        8/2009
Name: no_day_numeric, dtype: object

Random Sample After:
     match
423  0        12/1986
433  0        05/2006
365  0        07/1997
392  0        05/2000
453  0        07/1991
Name: no_day_numeric, dtype: object

Count of cleaned: 112
Random Sample Before:
     match
378  0        12/1978
447  0        07/1985
386  0        07/2009
360  0        12/2008
362  0        08/2003
Name: no_day_numeric, dtype: object

Random Sample After:
     match
389  0        02/01/2009
392  0        05/01/2000
356  0        09/01/2003
358  0        01/01/1983
377  0        06/01/2001
Name: no_day_numeric, dtype: object

Count of cleaned: 112


In [26]:
def clean_punctuation(source, sample=5):
    """removes punctuation

    Args:
     source (pandas.Series): data to clean
     sample (int): size of sample to show

    Returns:
     pandas.Series: source with punctuation removed
    """
    print("Cleaning Punctuation")
    if any(source.str.contains(PUNCTUATION)):
        source = source.str.replace(PUNCTUATION, EMPTY_STRING)
    return source


In [27]:
LONG_TO_SHORT = dict(January="Jan",
                     February="Feb",
                     March="Mar",
                     April="Apr",
                     May="May",
                     June="Jun",
                     July="Jul",
                     August="Aug",
                     September="Sep",
                     October="Oct",
                     November="Nov",
                     December="Dec")

# it turns out there are spelling errors in the data so this has to be fuzzy
LONG_TO_SHORT_EXPRESSION = OR.join([GROUP.format(month)
                                    + CLASS.format(LOWER_CASE)
                                    + ZERO_OR_MORE
                                    for month in LONG_TO_SHORT.values()])

def long_month_to_short(match):
    """convert long month to short
    
    Args:
     match (re.Match): object matching a long month

    Returns:
     str: shortened version of the month
    """
    return match.group(match.lastindex)


In [28]:
def convert_long_months_to_short(source, sample=5):
    """convert long month names to short
    
    Args:
     source (pandas.Series): data with months
     sample (int): size of sample to show

    Returns:
     pandas.Series: data with short months
    """
    return clean(source,
                 LONG_TO_SHORT_EXPRESSION,
                 long_month_to_short)


In [30]:
digits = ("{:02}".format(month) for month in range(1, 13))
MONTH_TO_DIGITS = dict(zip(MONTH_PREFIXES, digits))
SHORT_MONTHS_EXPRESSION = OR.join((GROUP.format(month) for month in MONTH_TO_DIGITS))
def month_to_digits(match):
    """converts short month to digits

    Args:
     match (re.Match): object with short-month

    Returns:
     str: month as two-digit number (e.g. Jan -> 01)
    """
    return MONTH_TO_DIGITS[match.group()]


In [31]:
def convert_short_month_to_digits(source, sample=5):
    """converts three-letter months to two-digits

    Args:
     source (pandas.Series): data with three-letter months
     sample (int): number of samples to show

    Returns:
     pandas.Series: source with short-months coverted to digits
    """
    return clean(source,
                 SHORT_MONTHS_EXPRESSION,
                 month_to_digits,
                 sample)


In [32]:
def clean_months(source, sample=5):
    """clean up months (which start as words)

    Args:
     source (pandas.Series): source of the months
     sample (int): number of random samples to show
    """
    cleaned = clean_punctuation(source)
    
    print("Converting long months to short")
    cleaned = clean(cleaned,
                    LONG_TO_SHORT_EXPRESSION,
                    long_month_to_short, sample)

    print("Converting short months to digits")
    cleaned = clean(cleaned,
                    SHORT_MONTHS_EXPRESSION,
                    month_to_digits, sample)
    return cleaned


In [34]:
no_day_cleaned = clean_months(no_day)
no_day_cleaned = clean(no_day_cleaned,
                       SPACE + ONE_OR_MORE,
                       lambda match: "/01/")


Cleaning Punctuation
Converting long months to short
Random Sample Before:
     match
248  0         July 1995
336  0        March 1978
318  0          Jan 1978
262  0          Jun 2002
249  0        April 1993
Name: no_day, dtype: object

Random Sample After:
     match
274  0        Apr 1985
303  0        Jan 1977
291  0        Jan 2004
268  0        Dec 2009
330  0        Apr 1988
Name: no_day, dtype: object

Count of cleaned: 115
Converting short months to digits
Random Sample Before:
     match
328  0        May 2001
301  0        Dec 1992
307  0        Jul 2006
311  0        Feb 1995
334  0        Jul 1986
Name: no_day, dtype: object

Random Sample After:
     match
282  0        05 2012
304  0        03 2002
291  0        01 2004
258  0        04 1976
255  0        10 2014
Name: no_day, dtype: object

Count of cleaned: 115
Random Sample Before:
     match
291  0        01 2004
309  0        04 1977
310  0        10 1992
306  0        05 2004
275  0        09 1984
Name: no_day, d

In [35]:
frame = pandas.DataFrame(backwards.str.split().tolist(),
                         columns="day month year".split())


In [38]:
frame.day = clean_two_digits(frame.day)
frame.month = clean_months(frame.month)
backwards_cleaned = frame.month + SLASH + frame.day + SLASH + frame.year
backwards_cleaned.index = backwards.index
print(backwards_cleaned.sample(5))


Random Sample Before:
0     24
58    26
60    17
40    18
10    10
Name: day, dtype: object

Random Sample After:
47    23
57    13
60    17
18    14
0     24
Name: day, dtype: object

Count of cleaned: 69
Cleaning Punctuation
Converting long months to short
Random Sample Before:
4     05
8     06
11    02
3     06
19    10
Name: month, dtype: object

Random Sample After:
48    10
61    10
31    05
50    10
51    05
Name: month, dtype: object

Count of cleaned: 69
Converting short months to digits
Random Sample Before:
50    10
45    01
23    02
59    11
66    11
Name: month, dtype: object

Random Sample After:
64    10
5     10
59    11
8     06
39    10
Name: month, dtype: object

Count of cleaned: 69
     match
181  0        08/18/1995
191  0        11/30/1972
129  0        05/06/1972
133  0        06/28/1994
165  0        08/18/1975
dtype: object


In [41]:
frame = pandas.DataFrame(words.str.split().tolist(), columns="month day year".split())
frame.month = clean_months(frame.month)
frame.day = clean_punctuation(frame.day)
words_cleaned = frame.month + SLASH + frame.day + SLASH + frame.year
words_cleaned.index = words.index
print(words_cleaned.sample(5))
                                                  

Cleaning Punctuation
Converting long months to short
Random Sample Before:
17     April
13    August
27       Oct
20       Sep
2        Feb
Name: month, dtype: object

Random Sample After:
7     Dec
33    Sep
28    May
2     Feb
24    May
Name: month, dtype: object

Count of cleaned: 34
Converting short months to digits
Random Sample Before:
13    Aug
3     Feb
4     Oct
12    Oct
15    Jul
Name: month, dtype: object

Random Sample After:
9     09
28    05
11    01
27    10
20    09
Name: month, dtype: object

Count of cleaned: 34
Cleaning Punctuation
     match
206  0        10/23/1990
227  0        09/15/2011
197  0        02/18/1981
201  0        12/23/1999
196  0        02/18/1994
dtype: object


In [42]:
cleaned = twentieth.str.replace(DASH, SLASH)
frame = pandas.DataFrame(cleaned.str.split(SLASH).tolist(),
                         columns=["month", "day", "year"])
frame.month = clean_two_digits_isolated(frame.month)
frame.day = clean_two_digits_isolated(frame.day)
frame.year = clean(frame.year, TWO_DIGITS, lambda match: "19" + match.group())
twentieth_cleaned = frame.month + SLASH + frame.day + SLASH + frame.year
twentieth_cleaned.index = twentieth.index
                                                  

Random Sample Before:
75     7
67     5
80    10
89    12
79     8
Name: month, dtype: object

Random Sample After:
72    10
53    10
40    02
81    09
88    10
Name: month, dtype: object

Count of cleaned: 100
Random Sample Before:
44    11
93    20
48    24
7     24
53    29
Name: day, dtype: object

Random Sample After:
55    21
50    22
17    21
5     06
47    01
Name: day, dtype: object

Count of cleaned: 100
Random Sample Before:
5     79
84    83
29    83
85    74
92    87
Name: year, dtype: object

Random Sample After:
27    1987
2     1971
22    1989
18    1979
26    1972
Name: year, dtype: object

Count of cleaned: 100


In [43]:
frame = pandas.DataFrame(numeric.str.split(SLASH).tolist(),
                         columns="month day year".split())
frame.month = clean_two_digits_isolated(frame.month)
frame.day = clean_two_digits_isolated(frame.day)
numeric_cleaned = frame.month + SLASH + frame.day + SLASH + frame.year
numeric_cleaned.index = numeric.index


Random Sample Before:
20     4
8      2
18    04
5      5
11    08
Name: month, dtype: object

Random Sample After:
10    12
19    12
21    07
24    04
11    08
Name: month, dtype: object

Count of cleaned: 25
Random Sample Before:
17    09
18    08
22    11
10     5
16    14
Name: day, dtype: object

Random Sample After:
7     13
17    09
11    20
2     13
23    20
Name: day, dtype: object

Count of cleaned: 25


In [46]:
cleaned = pandas.concat([numeric_cleaned,
                         twentieth_cleaned,
                         words_cleaned,
                         backwards_cleaned,
                         no_day_cleaned,
                         no_day_numeric_cleaned,
                         year_only_cleaned,
                         leftovers_cleaned,
])


In [47]:
datetimes = pandas.to_datetime(cleaned, format="%m/%d/%Y")

In [48]:
sorted_dates = datetimes.sort_values()

In [49]:
def date_sorter():
    return pandas.Series(sorted_dates.index.labels[0])